In [2]:
from mdutils.mdutils import MdUtils
import tqdm
import math
import requests
import pandas as pd
import time
from wikidataintegrator import wdi_core, wdi_login
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode
from rdflib.namespace import DCTERMS, FOAF, XSD, DC, RDFS
import shutil
import os

# Load occurence data

In [3]:
df = pd.read_csv('data/0094816-240506114902167/occurrence.txt', sep='\t', on_bad_lines = 'warn')
df

/var/folders/nt/bqljt7p919zgfjwhldlsbdvh0000gn/T/ipykernel_73160/4126030977.py:1: DtypeWarning: Columns (1,2,3,5,7,8,9,10,11,12,15,16,18,19,20,23,25,28,29,30,31,33,35,38,40,41,42,43,44,45,46,47,48,49,52,53,56,57,58,61,63,69,70,71,74,75,76,77,78,79,81,82,83,86,87,89,90,92,95,96,102,103,104,105,107,108,109,110,111,113,114,115,116,117,118,119,120,130,131,132,133,134,135,136,138,139,140,141,145,149,150,151,152,153,155,162,170,173,175,177,178,188,204,208,210,218,219) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/0094816-240506114902167/occurrence.txt', sep='\t', on_bad_lines = 'warn')


,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
0,657501415,http://vertnet.org/resources/norms.html,8715302b-da27-46cd-8057-27d5711b07c5. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8715...,NaN,PhysicalObject,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC
1,657404117,http://vertnet.org/resources/norms.html,0598e076-114d-4939-ba48-7aa485e4c820. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=0598...,NaN,PhysicalObject,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC
2,657404116,http://vertnet.org/resources/norms.html,8e616b5b-a0b0-4cef-9139-51475b0f2c01. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8e61...,NaN,PhysicalObject,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC
3,1927898036,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NORTH_AMERICA,MSR,Montserrat,MSR.1_1,Saint Anthon,NaN,NaN,NaN,NaN,NE
4,1927915415,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55349,1319774760,NaN,NaN,NaN,CC0_1_0,2016-09-21T02:29:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE
55350,1320213284,NaN,NaN,NaN,CC0_1_0,2018-09-24T10:06:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE
55351,1320226956,NaN,NaN,NaN,CC0_1_0,2017-07-19T14:43:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE
55352,1320517056,NaN,NaN,NaN,CC0_1_0,2022-03-24T10:31:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fetch institute QID's for institutes

In [4]:
user_agent = 'TaxonQueries/1.0 (andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['Institute', 'Institute_QID'])
batchSize = 5
retry_wait_time = 1  # Initial wait time in seconds
institutes = set(df["publisher"])
instituteQID = dict()
i = 0
notFound = {}
for institute in institutes:
    en = wdi_core.WDItemEngine.get_wd_search_results(institute, language='en')
    if len(en) > 0:
        instituteQID[institute] = en[0]
        continue
    nl = wdi_core.WDItemEngine.get_wd_search_results(institute, language='nl')
    if len(nl) > 0:
        instituteQID[institute] = nl[0]
        continue
    de = wdi_core.WDItemEngine.get_wd_search_results(institute, language='de')
    if len(de) > 0:
        instituteQID[institute] = de[0]
        continue
    fr = wdi_core.WDItemEngine.get_wd_search_results(institute, language='fr')
    if len(fr) > 0:
        instituteQID[institute] = fr[0]
        continue
    es = wdi_core.WDItemEngine.get_wd_search_results(institute, language='es')
    if len(es) > 0:
        instituteQID[institute] = es[0]
        continue
    pt = wdi_core.WDItemEngine.get_wd_search_results(institute, language='pt')
    if len(pt) > 0:
        instituteQID[institute] = pt[0]
        continue
    else:
        i+=1
        print(str(i)+f" Not found institute={institute}")
        notFound[institute] = requests.get("https://api.gbif.org/v1/organization?q="+institute).json()

1 Not found institute=Delaware Museum of Nature & Science
2 Not found institute=PANGAEA - Data Publisher for Earth & Environmental Science
3 Not found institute=W.S. Turrell Herbarium (MU)
4 Not found institute=Vermont Center for Ecostudies
5 Not found institute=University of Puerto Rico Mayagüez Invertebrate Collection
6 Not found institute=Plant Breeding and Acclimatization Institute (IHAR) - National Research Institute
7 Not found institute=UMS PatriNat (OFB-CNRS-MNHN), Paris
8 Not found institute=Walter Reed Biosystematics Unit, Smithsonian Institution
9 Not found institute=Lund University Biological Museum
10 Not found institute=Marion Ownbey Herbarium Washington State University


In [8]:
df2 = df[df["taxonRank"] == "SPECIES"]
df3 = df2[["species", "taxonKey", "taxonRank"]].drop_duplicates()
species_counts = df3["species"].value_counts()
species_counts

grouped = df3.groupby('species').apply(lambda x: list(x.taxonKey))
for species, taxon_keys in grouped.items():
    if len(taxon_keys) > 1:
        taxon_keys = list(map(int, taxon_keys))
        print(f"Species: {species}, Taxon Keys: {taxon_keys}")

Species: Acaciella glauca, Taxon Keys: [3794870, 2970419, 2970429, 2980038]
Species: Acmella uliginosa, Taxon Keys: [3092136, 5386720]
Species: Acropora palmata, Taxon Keys: [7767101, 8199465, 7731063]
Species: Actinia aster, Taxon Keys: [7675070, 4339194]
Species: Adiantum pyramidale, Taxon Keys: [2651842, 2651841]
Species: Agaricia agaricites, Taxon Keys: [11109477, 11003994]
Species: Allenia fusca, Taxon Keys: [2494941, 5231687]
Species: Allopathes desbonni, Taxon Keys: [2258277, 2258278]
Species: Allopeas micra, Taxon Keys: [8188096, 2294125]
Species: Amauropelta balbisii, Taxon Keys: [5275267, 11755245, 5275268]
Species: Amauropelta opposita, Taxon Keys: [5275260, 4080892, 7319365]
Species: Andira inermis, Taxon Keys: [2952148, 2952214]
Species: Anemia adiantifolia, Taxon Keys: [2650258, 10929199]
Species: Anomis argillacea, Taxon Keys: [4301049, 1784968]
Species: Anomocora fecunda, Taxon Keys: [2258936, 4339370]
Species: Antillotyphlops monastus, Taxon Keys: [9343898, 2471371]
Sp

/var/folders/nt/bqljt7p919zgfjwhldlsbdvh0000gn/T/ipykernel_53751/69063171.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df3.groupby('species').apply(lambda x: list(x.taxonKey))


In [9]:
taxonname =  df2[["species", "taxonKey", "taxonRank", "taxonRank", "genus", "genericName","taxonomicStatus", "acceptedNameUsageID", "scientificName", "specificEpithet"]].drop_duplicates()
taxonnames = []
for index, row in taxonname.iterrows():
    if isinstance(row["specificEpithet"], str):
        taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
    else:
        continue
    taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
taxonnames = list(set(taxonnames))

# resolve taxonnames and kingdom

In [10]:
url = 'https://query.wikidata.org/sparql'
user_agent = 'TaxonQueries/1.0 ( andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom' ])
batchSize = 10
retry_wait_time = 1  # Initial wait time in seconds

for i in tqdm.tqdm(range(math.ceil(len(taxonnames)/batchSize)), desc="Processing batches"):
    batch = taxonnames[i*batchSize:(i+1)*batchSize]
    query = """
            SELECT ?taxon ?taxonId ?taxonname ?kingdomname ?kingdom ?article
                WHERE {
                    VALUES ?taxonname {""" + " ".join(batch) + """}
                    ?taxon wdt:P225 ?taxonname .
                    ?taxon wdt:P171* ?kingdom .
                    OPTIONAL {?article schema:about ?taxon ;
                               schema:isPartOf <https://en.wikipedia.org/> .}
                    ?kingdom wdt:P105 wd:Q36732 .
                    ?kingdom rdfs:label ?kingdomname filter (lang(?kingdomname) = "en") .
                    OPTIONAL {?taxon wdt:P846 ?taxonId .}
            }
            """
    # print(query)

    success = False
    while not success:
        try:
            headers = {'User-Agent': user_agent}
            r = requests.get(url, params={'format': 'json', 'query': query}, headers=headers)
            r.raise_for_status()  # Raise an HTTPError for bad responses
            data = r.json()
            temp_df = pd.DataFrame([(item.get("taxonId", {}).get("value", ""),
                                     item.get("taxonname", {}).get("value", ""),
                                     item.get("taxon", {}).get("value", ""),
                                     item.get("kingdomname", {}).get("value", ""),
                                     item.get("kingdom", {}).get("value", ""),
                                     item.get("article", {}).get("value", "")
                                    )
                                    for item in data["results"]["bindings"]],
                                    columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom', 'article' ])

            # Concatenate the temporary DataFrame with the main result DataFrame
            result_df = pd.concat([result_df, temp_df], ignore_index=True)
            success = True  # Set to True to break out of the while loop on success

        except requests.exceptions.HTTPError as err:
            print(f"Error: {err}")
            print(f"Retrying in {retry_wait_time} seconds...")
            time.sleep(retry_wait_time)
            retry_wait_time *= 2  # Double the wait time
            if retry_wait_time > 60:  # Limit the maximum wait time to 60 seconds
                retry_wait_time = 60
        else:
            retry_wait_time = 1  # Reset to 1 second after a successful query

# Now, result_df contains all the data
result_df


Processing batches: 100%|██████████| 247/247 [03:17<00:00,  1.25it/s]


,taxonId,taxonname,taxon,kingdomname,kingdom,article
0,2278527,Asteroporpa annulata,http://www.wikidata.org/entity/Q3266837,animal,http://www.wikidata.org/entity/Q729,
1,3069048,Euphorbia pilulifera,http://www.wikidata.org/entity/Q21874235,plant,http://www.wikidata.org/entity/Q756,
2,7300703,Pilea deltoidea,http://www.wikidata.org/entity/Q110901302,plant,http://www.wikidata.org/entity/Q756,
3,,Eantis minor,http://www.wikidata.org/entity/Q119780041,animal,http://www.wikidata.org/entity/Q729,
4,5421132,Sapindus saponaria,http://www.wikidata.org/entity/Q2672802,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Sapindus_saponaria
...,...,...,...,...,...,...
2268,1314706,Monomorium ebeninum,http://www.wikidata.org/entity/Q13861227,animal,http://www.wikidata.org/entity/Q729,
2269,5382197,Phyllanthus stipulatus,http://www.wikidata.org/entity/Q15457785,plant,http://www.wikidata.org/entity/Q756,
2270,2689176,Odontoschisma sphagni,http://www.wikidata.org/entity/Q17283299,plant,http://www.wikidata.org/entity/Q756,
2271,2495696,Streptopelia decaocto,http://www.wikidata.org/entity/Q54696,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Eurasian_collare...


# Merge taxonnames and kingdom

In [11]:
result_df[['genericName', 'specificEpithet']] = result_df['taxonname'].str.split(expand=True, n=1)

# Print the updated DataFrame
wdtaxqid = dict()
for index, row in result_df.iterrows():
    if row["taxonId"] != "":
        wdtaxqid[row["taxonId"]] = row["taxon"]
result_df

,taxonId,taxonname,taxon,kingdomname,kingdom,article,genericName,specificEpithet
0,2278527,Asteroporpa annulata,http://www.wikidata.org/entity/Q3266837,animal,http://www.wikidata.org/entity/Q729,,Asteroporpa,annulata
1,3069048,Euphorbia pilulifera,http://www.wikidata.org/entity/Q21874235,plant,http://www.wikidata.org/entity/Q756,,Euphorbia,pilulifera
2,7300703,Pilea deltoidea,http://www.wikidata.org/entity/Q110901302,plant,http://www.wikidata.org/entity/Q756,,Pilea,deltoidea
3,,Eantis minor,http://www.wikidata.org/entity/Q119780041,animal,http://www.wikidata.org/entity/Q729,,Eantis,minor
4,5421132,Sapindus saponaria,http://www.wikidata.org/entity/Q2672802,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Sapindus_saponaria,Sapindus,saponaria
...,...,...,...,...,...,...,...,...
2268,1314706,Monomorium ebeninum,http://www.wikidata.org/entity/Q13861227,animal,http://www.wikidata.org/entity/Q729,,Monomorium,ebeninum
2269,5382197,Phyllanthus stipulatus,http://www.wikidata.org/entity/Q15457785,plant,http://www.wikidata.org/entity/Q756,,Phyllanthus,stipulatus
2270,2689176,Odontoschisma sphagni,http://www.wikidata.org/entity/Q17283299,plant,http://www.wikidata.org/entity/Q756,,Odontoschisma,sphagni
2271,2495696,Streptopelia decaocto,http://www.wikidata.org/entity/Q54696,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Eurasian_collare...,Streptopelia,decaocto


In [12]:
merged_df = pd.merge(df, result_df, on=['genericName', 'specificEpithet'], how='inner')
merged_df

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level2Name,level3Gid,level3Name,iucnRedListCategory,taxonId,taxonname,taxon,kingdomname,kingdom_y,article
0,657501415,http://vertnet.org/resources/norms.html,8715302b-da27-46cd-8057-27d5711b07c5. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8715...,NaN,PhysicalObject,...,NaN,NaN,NaN,LC,2459658,Iguana iguana,http://www.wikidata.org/entity/Q215376,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Green_iguana
1,657404117,http://vertnet.org/resources/norms.html,0598e076-114d-4939-ba48-7aa485e4c820. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=0598...,NaN,PhysicalObject,...,NaN,NaN,NaN,LC,2467075,Anolis bimaculatus,http://www.wikidata.org/entity/Q304822,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_bimaculatus
2,657404116,http://vertnet.org/resources/norms.html,8e616b5b-a0b0-4cef-9139-51475b0f2c01. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8e61...,NaN,PhysicalObject,...,NaN,NaN,NaN,LC,2467075,Anolis bimaculatus,http://www.wikidata.org/entity/Q304822,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_bimaculatus
3,1927898036,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NaN,NaN,NaN,NE,7313869,Freziera undulata,http://www.wikidata.org/entity/Q15395965,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Freziera_undulata
4,1927915415,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NaN,NaN,NaN,LC,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53320,1319770923,NaN,NaN,NaN,CC0_1_0,2018-10-02T09:49:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NaN,NT,2467434,Anolis lividus,http://www.wikidata.org/entity/Q1445510,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_lividus
53321,1319774760,NaN,NaN,NaN,CC0_1_0,2016-09-21T02:29:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NaN,NE,2650883,Hemidictyum marginatum,http://www.wikidata.org/entity/Q2383295,plant,http://www.wikidata.org/entity/Q756,
53322,1320213284,NaN,NaN,NaN,CC0_1_0,2018-09-24T10:06:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NaN,NE,3780480,Justicia eustachiana,http://www.wikidata.org/entity/Q18107440,plant,http://www.wikidata.org/entity/Q756,
53323,1320226956,NaN,NaN,NaN,CC0_1_0,2017-07-19T14:43:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NaN,NE,2984573,Artocarpus altilis,http://www.wikidata.org/entity/Q14677,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Breadfruit


In [13]:
merged_df[['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet', 'kingdom_y', 'kingdomname', 'article']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet,kingdom_y,kingdomname,article
0,2459658,2459658,Iguana iguana,http://www.wikidata.org/entity/Q215376,Iguana,iguana,http://www.wikidata.org/entity/Q729,animal,https://en.wikipedia.org/wiki/Green_iguana
1,2467075,2467075,Anolis bimaculatus,http://www.wikidata.org/entity/Q304822,Anolis,bimaculatus,http://www.wikidata.org/entity/Q729,animal,https://en.wikipedia.org/wiki/Anolis_bimaculatus
3,7313869,7313869,Freziera undulata,http://www.wikidata.org/entity/Q15395965,Freziera,undulata,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Freziera_undulata
4,5421109,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,Sauvagesia,erecta,http://www.wikidata.org/entity/Q756,plant,
5,2970418,2970418,Leucaena leucocephala,http://www.wikidata.org/entity/Q35905,Leucaena,leucocephala,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Leucaena_leucoce...
...,...,...,...,...,...,...,...,...,...
53284,5190198,5190198,Gaza superba,http://www.wikidata.org/entity/Q3175317,Gaza,superba,http://www.wikidata.org/entity/Q729,animal,https://en.wikipedia.org/wiki/Gaza_superba
53299,2925467,2925466,Aegiphila martinicensis,http://www.wikidata.org/entity/Q15339440,Aegiphila,martinicensis,http://www.wikidata.org/entity/Q756,plant,
53300,2650646,2650646,Asplenium laetum,http://www.wikidata.org/entity/Q15601691,Asplenium,laetum,http://www.wikidata.org/entity/Q756,plant,
53301,2304126,2304126,Murexiella hidalgoi,http://www.wikidata.org/entity/Q49505930,Murexiella,hidalgoi,http://www.wikidata.org/entity/Q729,animal,


In [14]:
taxoninformation = dict()

In [15]:
def get_wikipedia_snippet(title):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True
    }
    response = requests.get(url, params=params)
    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    snippet = page.get("extract", "")
    return snippet



In [18]:
# Assuming `result_df` is your DataFrame and it has columns named 'article' and 'taxonname'
taxonname_list = result_df[result_df["article"] != ""]["taxonname"].tolist()
taxonname_list

['Sapindus saponaria',
 'Anchoa hepsetus',
 'Henricia sexradiata',
 'Aristida adscensionis',
 'Lepidagathis alopecuroidea',
 'Eclipta prostrata',
 'Pilea microphylla',
 'Mangifera indica',
 'Calidris pusilla',
 'Charonia variegata',
 'Passiflora rubra',
 'Calidris minuta',
 'Tradescantia zebrina',
 'Carissa macrocarpa',
 'Cleome gynandra',
 'Pycnoscelus surinamensis',
 'Polistes crinitus',
 'Plumbago auriculata',
 'Rigidoporus lineatus',
 'Handroanthus serratifolius',
 'Volvarina rubella',
 'Patagioenas leucocephala',
 'Canavalia rosea',
 'Loxigilla noctis',
 'Passiflora suberosa',
 'Garcinia humilis',
 'Huttonella bicolor',
 'Palicourea padifolia',
 'Coffea arabica',
 'Crescentia cujete',
 'Urbanus proteus',
 'Chromolaena sinuata',
 'Ceiba pentandra',
 'Mimosa pudica',
 'Cantherhines pullus',
 'Amaranthus viridis',
 'Sula dactylatra',
 'Manicina areolata',
 'Cyperus odoratus',
 'Calidris himantopus',
 'Neritilia succinea',
 'Famelica scipio',
 'Terminalia buceras',
 'Anacardium occide

In [19]:
query = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?taxon  WHERE {
           ?observation wdt:P225 ?taxon .
           ?taxon rdfs:label ?taxonLabel .
            }"""
# results = gbifRDF.query(query)
for taxonname in tqdm.tqdm(taxonname_list):
    if taxonname not in taxoninformation.keys():
        taxoninformation[taxonname] = dict()
        taxoninformation[taxonname]["wikiintroduction"] = get_wikipedia_snippet(taxonname.replace(" ", "_"))


100%|██████████| 1069/1069 [05:22<00:00,  3.31it/s]


In [20]:
taxon_info_df = pd.DataFrame.from_dict(taxoninformation, orient='index').reset_index()
taxon_info_df.rename(columns={'index': 'taxonname'}, inplace=True)

# Merge the dictionary DataFrame with your existing DataFrame
merged_df2 = merged_df.merge(taxon_info_df, on='taxonname', how='left')
merged_df2

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level3Gid,level3Name,iucnRedListCategory,taxonId,taxonname,taxon,kingdomname,kingdom_y,article,wikiintroduction
0,657501415,http://vertnet.org/resources/norms.html,8715302b-da27-46cd-8057-27d5711b07c5. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8715...,NaN,PhysicalObject,...,NaN,NaN,LC,2459658,Iguana iguana,http://www.wikidata.org/entity/Q215376,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Green_iguana,
1,657404117,http://vertnet.org/resources/norms.html,0598e076-114d-4939-ba48-7aa485e4c820. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=0598...,NaN,PhysicalObject,...,NaN,NaN,LC,2467075,Anolis bimaculatus,http://www.wikidata.org/entity/Q304822,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_bimaculatus,"Anolis bimaculatus, the panther anole, also kn..."
2,657404116,http://vertnet.org/resources/norms.html,8e616b5b-a0b0-4cef-9139-51475b0f2c01. LACM Ver...,en,CC0_1_0,2024-04-26T00:00:00Z,Natural History Museum of Los Angeles County,http://portal.vertnet.org/o/lacm/herps?id=8e61...,NaN,PhysicalObject,...,NaN,NaN,LC,2467075,Anolis bimaculatus,http://www.wikidata.org/entity/Q304822,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_bimaculatus,"Anolis bimaculatus, the panther anole, also kn..."
3,1927898036,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NaN,NaN,NE,7313869,Freziera undulata,http://www.wikidata.org/entity/Q15395965,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Freziera_undulata,Freziera undulata is a species of flowering pl...
4,1927915415,NaN,NaN,en,CC0_1_0,NaN,The New York Botanical Garden,http://sweetgum.nybg.org/science/vh/specimen_d...,The New York Botanical Garden,PhysicalObject,...,NaN,NaN,LC,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53320,1319770923,NaN,NaN,NaN,CC0_1_0,2018-10-02T09:49:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NT,2467434,Anolis lividus,http://www.wikidata.org/entity/Q1445510,animal,http://www.wikidata.org/entity/Q729,https://en.wikipedia.org/wiki/Anolis_lividus,"Anolis lividus, the Plymouth anole or Montserr..."
53321,1319774760,NaN,NaN,NaN,CC0_1_0,2016-09-21T02:29:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NE,2650883,Hemidictyum marginatum,http://www.wikidata.org/entity/Q2383295,plant,http://www.wikidata.org/entity/Q756,,NaN
53322,1320213284,NaN,NaN,NaN,CC0_1_0,2018-09-24T10:06:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NE,3780480,Justicia eustachiana,http://www.wikidata.org/entity/Q18107440,plant,http://www.wikidata.org/entity/Q756,,NaN
53323,1320226956,NaN,NaN,NaN,CC0_1_0,2017-07-19T14:43:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,NaN,NE,2984573,Artocarpus altilis,http://www.wikidata.org/entity/Q14677,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Breadfruit,


In [21]:
gbifRDF = Graph()
WD = Namespace("http://www.wikidata.org/entity/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
gbifRDF.bind("wd", WD)
gbifRDF.bind("wdt", WDT)
gbifRDF.bind("dcterms", DCTERMS)
gbifRDF.bind("geo", Namespace("http://www.opengis.net/ont/geosparql#"))

for index, row in tqdm.tqdm(merged_df2.iterrows()):
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    gbifRDF.add((GBIFURI, WDT.P854 , URIRef("https://www.gbif.org/occurrence/"+str(row["gbifID"]))))
    gbifRDF.add((GBIFURI, DCTERMS.identifier , Literal(str(row["gbifID"]))))

    # species
    tempkey = str(row['taxonKey']).replace('.0', '')
    if tempkey in wdtaxqid.keys():
        gbifRDF.add((GBIFURI, WDT.P225, URIRef(wdtaxqid[tempkey])))
        gbifRDF.add((URIRef(wdtaxqid[tempkey]), RDFS.label, Literal(row["taxonname"], lang="en")))

    # lat long
    try:
        if not math.isnan(float(row["decimalLatitude"])):
            gbifRDF.add((GBIFURI, WDT.P625, Literal("Point("+str(row["decimalLongitude"])+" "+str(row["decimalLatitude"])+")", datatype="http://www.opengis.net/ont/geosparql#wktLiteral")))
    except (TypeError, ValueError) as e:
        print(f"Error for gbifID={row['gbifID']}, Latitude={row['decimalLatitude']}, Longitude={row['decimalLongitude']}")
        print(f"Error details: {e}")
        print(f"Error occurred in this line: {e.__traceback__.tb_lineno}")

    # type
    typeDict = {
    "Collection": "Q2668072","Colletion": "Q2668072","collection": "Q2668072",
    "Event": "Q1656682",
    "Objeto físico": "Q223557","Physical Object": "Q223557","Physical object": "Q223557","PhysicalObject": "Q223557","PhysicalSpecimen": "Q223557","http://purl.org/dc/dcmitype/PhysicalObject": "Q223557",
    "Sound": "Q11442",
    "StillImage": "Q478798","http://purl.org/dc/dcmitype/StillImage": "Q478798",
    "Text": "Q1145976","text": "Q1145976"
}
    if row["type"] in typeDict.keys():
        gbifRDF.add((GBIFURI, WDT.P31, URIRef("http://www.wikidata.org/entity/"+typeDict[row["type"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, type={row['type']}")

    # language
    langDict = {
        "EN": "Q1860", "en": "Q1860","En": "Q1860", "English": "Q1860",
        "PT": "Q5146","português": "Q5146",
        "es": "Q1321",
        "nl": "Q7411",
        "fr": "Q150", "FR": "Q150",
        "Latin": "Q397", "la": "Q397"
    }

    if not pd.isna(row["language"]) and row["language"] in langDict:
        gbifRDF.add((GBIFURI, WDT.P407, URIRef("http://www.wikidata.org/entity/"+langDict[row["language"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, language={row['language']}")

    # IUCN Red List Category
    iucnDict = {
        "NE": "Q3350324", "Not Evaluated": "Q3350324",
        "LC": "Q211005", "Least Concern": "Q211005",
        "NT": "Q719675", "Near Threatened": "Q719675",
        "VU": "Q278113", "Vulnerable": "Q278113",
        "EN": "Q11394", "Endangered": "Q11394",
        "CR": "Q219127", "Critically Endangered": "Q219127",
        "EW": "Q239509", "Extinct in the Wild": "Q239509",
        "EX": "Q237350", "Extinct": "Q237350",
        "DD": "Q3245245", "Data Deficient": "Q3245245"
    }
    if not pd.notna(row["iucnRedListCategory"]) and row["iucnRedListCategory"] in iucnDict:
        gbifRDF.add((GBIFURI, WDT.P141, URIRef("http://www.wikidata.org/entity/"+iucnDict[row["iucnRedListCategory"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, iucnRedListCategory={row['iucnRedListCategory']}")

    # taxonank
    taxonrankDict = {
        "class": "Q37517", "Class": "Q37517",
        "family": "Q35409", "Family": "Q35409",
        "genus": "Q34740", "Genus": "Q34740",
        "kingdom": "Q36732", "Kingdom": "Q36732",
        "order": "Q36602", "Order": "Q36602",
        "phylum": "Q38348", "Phylum": "Q38348",
        "species": "Q7432", "Species": "Q7432",
        "subspecies": "Q68947", "Subspecies": "Q68947",
        "variety": "Q767728", "Variety": "Q767728",
        "form": "Q279749", "Form": "Q279749"
    }

    if not pd.isna(row["taxonRank"]) and row["taxonRank"] in taxonrankDict:
        gbifRDF.add((GBIFURI, WDT.P105, URIRef("http://www.wikidata.org/entity/"+taxonrankDict[row["taxonRank"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, taxonRank={row['taxonRank']}")
    # license
    licenseDict = {
        "CC_BY_NC_ND_2_5": "Q19113746", "http://creativecommons.org/licenses/by-nc-nd/2.5/": "Q19113746",
        "CC_BY_NC_ND_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc-nd/4.0/": "Q24082749",
        "CC_BY_NC_SA_2_5": "Q19068212", "http://creativecommons.org/licenses/by-nc-sa/2.5/": "Q19068212",
        "CC_BY_NC_SA_3_0": "Q15643954", "http://creativecommons.org/licenses/by-nc-sa/3.0/": "Q15643954",
        "CC_BY_NC_SA_4_0": "Q42553662", "http://creativecommons.org/licenses/by-nc-sa/4.0/": "Q42553662",
        "CC_BY_NC_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc/4.0/": "Q24082749",
        "CC_BY_NC_3_0": "Q18810331", "http://creativecommons.org/licenses/by-nc/3.0/": "Q18810331",
        "CC_BY_ND_2_5": "Q18810338", "http://creativecommons.org/licenses/by-nd/2.5/": "Q18810338",
        "CC_BY_ND_3_0": "Q18810160", "http://creativecommons.org/licenses/by-nd/3.0/": "Q18810160",
        "CC_BY_ND_4_0": "Q36795408", "http://creativecommons.org/licenses/by-nd/4.0/": "Q36795408",
         "CC_BY_SA_3_0": "Q14946043","http://creativecommons.org/licenses/by-sa/3.0/": "Q14946043",
        "CC_BY_SA_4_0": "Q18199165", "http://creativecommons.org/licenses/by-sa/4.0/": "Q18199165",
        "CC_BY_4_0": "Q18199165", "http://creativecommons.org/licenses/by/4.0/": "Q18199165",
        "CC_BY_3_0": "Q14947546", "http://creativecommons.org/licenses/by/3.0/": "Q14947546",
        "CC0_1_0": "Q18199165", "http://creativecommons.org/publicdomain/zero/1.0/": "Q18199165",
        "https://huh.harvard.edu/access-digital-reproductions-works-public-domain":"Q18199165"
    }
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((GBIFURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, license={row['license']}")

    # sex
    sexDict = {
        "FEMALE": "Q43445",
        "MALE": "Q44148"
    }
    if not pd.isna(row["sex"]) and row["sex"] in sexDict:
        gbifRDF.add((GBIFURI, WDT.P21, URIRef("http://www.wikidata.org/entity/"+sexDict[row["sex"]])))
    else:
        pass

    # recordedByID
    if not pd.isna(row["recordedByID"]):
        gbifRDF.add((GBIFURI, WDT.P170, Literal(str(row["recordedByID"]), datatype=XSD.string)))
    else:
        pass

    # associatedSequences
    if not pd.isna(row["associatedSequences"]):
        gbifRDF.add((GBIFURI, WDT.P4333, Literal(str(row["associatedSequences"]), datatype=XSD.string)))
    else:
        pass

    # occurrenceID
    if not pd.isna(row["occurrenceID"]):
        gbifRDF.add((GBIFURI, DC.identifier, Literal(str(row["occurrenceID"]), datatype=XSD.string)))

    # publisher
    if row["publisher"] in instituteQID.keys():
        gbifRDF.add((GBIFURI, WDT.P123, URIRef("http://www.wikidata.org/entity/"+instituteQID[row["publisher"]])))
        gbifRDF.add((URIRef("http://www.wikidata.org/entity/"+instituteQID[row["publisher"]]), RDFS.label, Literal(row["publisher"], lang="en")))

    # kingfom
    if not pd.isna(row["kingdom_y"]):
        gbifRDF.add((GBIFURI, WDT.P910, URIRef(row["kingdom_y"]))) #chapter
        gbifRDF.add((URIRef(row["kingdom_y"]), RDFS.label, Literal(row["kingdomname"], lang="en")))
    else:
        pass

    #Wiki article
    if not pd.isna(row["article"]):
        gbifRDF.add((GBIFURI, DC.description, URIRef(row["article"]))) #chapter
    else:
        pass

53325it [00:16, 3328.67it/s]


In [24]:
df_image = pd.read_csv('data/0094816-240506114902167/multimedia.txt', sep='\t', on_bad_lines = 'warn')
for index, row in df_image.iterrows():
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    mediaURI = BNode()
    if row["type"] == "StillImage":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q17538176))
    elif row["type"] == "Sound":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q11461))
    gbifRDF.add((mediaURI, DCTERMS.format, Literal(row["format"], datatype=XSD.string)))
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((mediaURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
    gbifRDF.add((mediaURI, WDT.P2699, Literal(row["identifier"], datatype=XSD.string)))
    gbifRDF.add((mediaURI, WDT.P361, GBIFURI))

In [25]:
wdtaxqid

{'2278527': 'http://www.wikidata.org/entity/Q3266837',
 '3069048': 'http://www.wikidata.org/entity/Q21874235',
 '7300703': 'http://www.wikidata.org/entity/Q110901302',
 '5421132': 'http://www.wikidata.org/entity/Q2672802',
 '2413913': 'http://www.wikidata.org/entity/Q2926560',
 '5187323': 'http://www.wikidata.org/entity/Q3202890',
 '2706277': 'http://www.wikidata.org/entity/Q4790962',
 '3172921': 'http://www.wikidata.org/entity/Q7697291',
 '5384950': 'http://www.wikidata.org/entity/Q857092',
 '5361841': 'http://www.wikidata.org/entity/Q3023762',
 '3190638': 'http://www.wikidata.org/entity/Q3919027',
 '2275635': 'http://www.wikidata.org/entity/Q2554846',
 '3544512': 'http://www.wikidata.org/entity/Q10531157',
 '4156734': 'http://www.wikidata.org/entity/Q15332227',
 '3833580': 'http://www.wikidata.org/entity/Q15394845',
 '9922917': 'http://www.wikidata.org/entity/Q81904540',
 '4160015': 'http://www.wikidata.org/entity/Q111724119',
 '1920579': 'http://www.wikidata.org/entity/Q2171560',
 '

In [26]:
# Define the URI prefix you are interested in
prefix = "http://www.wikidata.org/entity/"

# Initialize a set to hold all matching URIs
wikidata_uris = set()

# Iterate through all nodes in the graph, both subjects and objects
for node in gbifRDF.all_nodes():
    if isinstance(node, URIRef) and str(node).startswith(prefix):
        wikidata_uris.add(str(node))


In [54]:
gbifRDF.serialize(destination='gbifMontserrat.ttl', format='turtle', encoding="utf-8")

<Graph identifier=N015c5699a1834fccab9016545983a6d6 (<class 'rdflib.graph.Graph'>)>

In [28]:
wdGraph = Graph()

# Wrapping wikidata_uris with tqdm to create a progress bar
for wduri in tqdm.tqdm(wikidata_uris, desc="Processing URIs"):
    try:
        print(str(wduri))
        wdGraph.parse(str(wduri), format='turtle')
    except Exception as e:  # It's a good practice to log the exception
        print(f"An error occurred while parsing {wduri}: {e}")
        continue

Processing URIs:   0%|          | 0/2221 [00:00<?, ?it/s]

http://www.wikidata.org/entity/Q3735054


Processing URIs:   0%|          | 1/2221 [00:01<57:44,  1.56s/it]

http://www.wikidata.org/entity/Q6756682


Processing URIs:   0%|          | 2/2221 [00:03<1:08:53,  1.86s/it]

...

Processing URIs: 100%|█████████▉| 2219/2221 [1:24:22<00:04,  2.37s/it]

http://www.wikidata.org/entity/Q814421


Processing URIs: 100%|█████████▉| 2220/2221 [1:24:24<00:02,  2.31s/it]

http://www.wikidata.org/entity/Q55863307


Processing URIs: 100%|██████████| 2221/2221 [1:24:25<00:00,  2.28s/it]


In [29]:
taxonpages = dict()
query = """
PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT  ?taxon ?taxonLabel ?publisher ?publisherLabel ?observation ?gbifObservation ?media ?media_url ?license WHERE {
           ?media wdt:P2699 ?media_url ;
                  wdt:P275 ?license ;
                  wdt:P361 ?observation .
          ?observation wdt:P225 ?taxon ;
                       wdt:P854 ?gbifObservation ;
                       wdt:P123 ?publisher .
          ?taxon rdfs:label ?taxonLabel .
          OPTIONAL {?taxon dc:description ?article .}
          ?publisher rdfs:label ?publisherLabel .
            }"""
# Assuming gbifRDF.query(query) has been defined and executed
results = gbifRDF.query(query)
taxonpages = {}  # Ensure that taxonpages is defined before the loop

for row in results:
    # Cache commonly used row items to avoid repetitive dictionary lookups and conversions
    taxon_label = str(row["taxonLabel"])
    publisher_label = str(row["publisherLabel"])

    # If the taxon label is not in taxonpages, initialize it with necessary keys
    if taxon_label not in taxonpages:
        taxonpages[taxon_label] = {
            "wd_uri": str(row["taxon"]),
            "publisher": {},  # Initialize the publisher dictionary here
        }
        # Add the article if it exists in the row
        if "article" in row:
            taxonpages[taxon_label]["article"] = str(row["article"])

    # Ensure the publisher label is initialized in the taxon's publisher dictionary
    if publisher_label not in taxonpages[taxon_label]["publisher"]:
        taxonpages[taxon_label]["publisher"][publisher_label] = {
            "observations": []  # Initialize the observations list here
        }

    # Construct the observation dictionary
    observation = {
        "observation_id": str(row["gbifObservation"]),
        "media": [str(row["media_url"])],  # Initialize media as a list with the media_url
        "license": str(row["license"])
    }
    # Append the observation dictionary to the observations list
    taxonpages[taxon_label]["publisher"][publisher_label]["observations"].append(observation)

# The rest of your processing can continue after this




In [30]:
taxonpages

{'Tetrodon rostratus': {'wd_uri': 'http://www.wikidata.org/entity/Q107054493',
  'publisher': {'Natural History Museum': {'observations': [{'observation_id': 'https://www.gbif.org/occurrence/1055544492',
      'media': ['https://data.nhm.ac.uk/media/e79134bb-3de6-4358-a912-27a63e77e18b'],
      'license': 'http://www.wikidata.org/entity/Q18199165'}]}}},
 'Cubaris murina': {'wd_uri': 'http://www.wikidata.org/entity/Q2461198',
  'publisher': {'Natural History Museum': {'observations': [{'observation_id': 'https://www.gbif.org/occurrence/1055563063',
      'media': ['https://data.nhm.ac.uk/media/5b9b3c0e-8aa7-4da1-96bb-ca25b760121c'],
      'license': 'http://www.wikidata.org/entity/Q18199165'}]}}},
 'Solanum seaforthianum': {'wd_uri': 'http://www.wikidata.org/entity/Q3024467',
  'publisher': {'Natural History Museum': {'observations': [{'observation_id': 'https://www.gbif.org/occurrence/1055646381',
      'media': ['https://data.nhm.ac.uk/media/5e0a83e6-2396-49c6-a7b1-d4f12321185b'],
   

In [33]:
import shutil
import os
shutil.rmtree('./taxa/')
os.makedirs(os.path.dirname('./taxa/'), exist_ok=True)


In [34]:
for taxon in taxonpages.keys():
    mdFile = MdUtils(file_name='taxa/'+ taxon.replace(" ", "_"), title=taxon+" ("+taxonpages[taxon]["wd_uri"].replace("http://www.wikidata.org/entity/", "")+")")

    if URIRef(row["taxon"]) in taxoninformation.keys():
        if "wikiintroduction" in taxoninformation[URIRef(row["taxon"])].keys():
            mdFile.new_line(taxoninformation[URIRef(row["taxon"])]["wikiintroduction"])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('' , '_')})")
    mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{taxonpages[taxon]['wd_uri'].replace('http://www.wikidata.org/entity/', '')})")
    for organisation in taxonpages[taxon]["publisher"].keys():
        mdFile.new_header(level=1, title=organisation)
        for observation in taxonpages[taxon]["publisher"][organisation]["observations"]:
            mdFile.new_line(f"Observation: [{observation['observation_id']}]({observation['observation_id']})")
            mdFile.new_line(f"License: [{observation['license']}]({observation['license']})")
            for media in observation["media"]:
                if media != "nan":
                    mdFile.new_line(f"![{observation['observation_id']}]({media.replace('square', 'medium')})")
    mdFile.create_md_file()


In [ ]:
organization_to_taxa = {}

# Populate the organization_to_taxa dictionary
for taxon, data in taxonpages.items():
    for organisation, org_data in data["publisher"].items():
        if organisation not in organization_to_taxa:
            organization_to_taxa[organisation] = []
        organization_to_taxa[organisation].append({
            "taxon": taxon,
            "taxon_uri": data["wd_uri"],
            "observations": org_data["observations"]
        })

# Generate md files for each organization
for organisation, taxa in organization_to_taxa.items():
    mdFile = MdUtils(file_name='organisation/' + organisation.replace(" ", "_"), title=organisation)

    for taxon_data in taxa:
        taxon = taxon_data["taxon"]
        taxon_uri = taxon_data["taxon_uri"]
        observations = taxon_data["observations"]
        
        mdFile.new_header(level=1, title=taxon + " (" + taxon_uri.replace("http://www.wikidata.org/entity/", "") + ")")
        
        if URIRef(taxon_uri) in taxoninformation:
            taxon_info = taxoninformation[URIRef(taxon_uri)]
            if "wikiintroduction" in taxon_info:
                mdFile.new_line(taxon_info["wikiintroduction"])
                mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{taxon.replace(' ', '_')})")
        
        mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{taxon_uri.replace('http://www.wikidata.org/entity/', '')})")
        
        for observation in observations:
            mdFile.new_line(f"Observation: [{observation['observation_id']}]({observation['observation_id']})")
            mdFile.new_line(f"License: [{observation['license']}]({observation['license']})")
            for media in observation["media"]:
                if media != "nan":
                    mdFile.new_line(f"![{observation['observation_id']}]({media.replace('square', 'medium')})")

    mdFile.create_md_file()

In [1]:
import yaml

with open(r'_toc.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    species = yaml.load(file, Loader=yaml.FullLoader)

species["parts"][0]["chapters"] = []
for file in os.listdir('./organisation/'):
    species["parts"][0]["chapters"].append({'file': 'organisation/'+file})

species["parts"][1]["chapters"] = []
for file in os.listdir('./taxa/'):
    species["parts"][1]["chapters"].append({'file': 'taxa/'+file})
with open(r'_toc.yml', 'w+') as file:
    documents = yaml.dump(species, file)

NameError: name 'os' is not defined

## Below is for reference only

In [37]:
# Assuming `notFound` is your dictionary
data = []  # List to hold data for DataFrame

for institute, details in notFound.items():
    try:
        # Access the first result for the current institute
        if len(details["results"]) == 0:
            # If no results are found, append a dictionary with the institute and an error message
            print(f"No results found for {institute}")
            data.append({
                "Institute": "FOUT" + institute,
                "Error": "No results found"
            })
            continue
        first_result = details["results"][0]

        # Append a dictionary with the required information to the data list
        data.append({
            "Institute": institute,
            "Key": first_result.get("key", None),
            "Title": first_result.get("title", None),
            "Homepage": first_result.get("homepage", None),
            "City": first_result.get("city", None),
            "Country": first_result.get("country", None),
            "PostalCode": first_result.get("postalCode", None),
            "Address": first_result.get("address", None),
            "Phone": first_result.get("phone", None),
            "Email": first_result.get("email", None),
            "Endpoints": first_result.get("endpoints", None)
        })
    except KeyError as e:
        # If some data is not found, append a dictionary with the institute and an error message
        print(f"Error retrieving data for {institute}: {e}")
        data.append({
            "Institute": "FOUT" + institute,
            "Error": details["results"]
        })

# Create DataFrame from the data list
dfinstitutestoBuild = pd.DataFrame(data)
dfinstitutestoBuild


,Institute,Key,Title,Homepage,City,Country,PostalCode,Address,Phone,Email,Endpoints
0,PANGAEA - Data Publisher for Earth & Environme...,d5778510-eb28-11da-8629-b8a03c50a862,PANGAEA - Data Publisher for Earth & Environme...,[http://www.pangaea.de],Bremerhaven,DE,None,[],[],[],[]
1,Lund University Biological Museum,c0758a16-296b-4ec3-97fd-8a057d9060f3,Lund University Biological Museum,[https://www.biology.lu.se/biological-museum],Lund,SE,22362,[Sölvegatan 37],[+46462223102],[niklas.wahlberg@biol.lu.se],[]
2,University of Puerto Rico Mayagüez Invertebrat...,45346344-bad3-4a45-8563-98955464b3ca,University of Puerto Rico Mayagüez Invertebrat...,[http://biology.uprm.edu/],Mayagüez,PR,00681-9000,"[Attn: Dr. Alex Van Dam, Departamento de Biolo...",[],[],[]
3,"Walter Reed Biosystematics Unit, Smithsonian I...",5ab24120-de25-11dc-9a41-b8a03c50a862,"Walter Reed Biosystematics Unit, Smithsonian I...","[http://worldviewsolutions.net:8080, https://w...",Suitland,US,20746-2863,[4210 Silver Hill Road],[],[nmnh-wrbu@si.edu],[]
4,"UMS PatriNat (OFB-CNRS-MNHN), Paris",1928bdf0-f5d2-11dc-8c12-b8a03c50a862,"UMS PatriNat (OFB-CNRS-MNHN), Paris",[http://www.patrinat.fr/fr],Paris,FR,None,[],[],[],[]
5,W.S. Turrell Herbarium (MU),b81636e9-2a0d-4020-a7a7-6e13f2721db1,W.S. Turrell Herbarium (MU),[],Oxford,US,45056,"[79 Upham Hall, Miami University]",[5135292755],[vincenma@miamioh.edu],[]
6,Marion Ownbey Herbarium Washington State Unive...,df281c91-b391-4154-8cf9-5185d4e6715f,Marion Ownbey Herbarium Washington State Unive...,[https://ownbeyherbarium.weebly.com/],Pullman,US,99164-4236,"[Marion Ownbey Herbarium, School of Biological...",[],[CAS.Herbarium@wsu.edu],[]
7,Delaware Museum of Nature & Science,a42867f0-813a-11dc-82e0-b8a03c50a862,Delaware Museum of Nature & Science,[http://www.delmns.org],Wilmington,US,19807,[],[],[],[]
8,Vermont Center for Ecostudies,b6d09100-919d-4026-b35b-22be3dae7156,Vermont Center for Ecostudies,[http://www.vtecostudies.org/],Norwich,US,05055,[PO Box 420],[802-245-4008],[info@vtecostudies.org],[]
9,Plant Breeding and Acclimatization Institute (...,24c3dc90-822b-11d8-b7ed-b8a03c50a862,Plant Breeding and Acclimatization Institute (...,[http://www.ihar.edu.pl],Radzików,PL,None,[],[],[],[]


In [38]:
df.groupby('license').count()

,gbifID,accessRights,bibliographicCitation,language,modified,publisher,references,rightsHolder,type,institutionID,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
license,,,,,,,,,,,,,,,,,,,,,
CC0_1_0,7430,600,155,1952,5315,7430,3197,3123,1908,5489,...,7430,1870,1870,1870,1870,25,25,0,0,6714
CC_BY_4_0,45487,445,95,896,1585,45487,310,243,577,994,...,45457,33223,33223,33223,33223,1,1,0,0,44821
CC_BY_NC_4_0,2437,340,38,332,2358,2437,1932,2221,371,1045,...,2435,1745,1745,1745,1745,0,0,0,0,2097


In [39]:
df.groupby('iucnRedListCategory').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,gbifRegion,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
iucnRedListCategory,,,,,,,,,,,,,,,,,,,,,
CR,127,36,2,18,127,99,127,41,37,22,...,127,127,17,17,17,17,0,0,0,0
DD,33,0,1,3,33,23,33,16,13,3,...,33,33,11,11,11,11,0,0,0,0
EN,24,2,2,4,24,14,24,7,5,5,...,24,24,2,2,2,2,0,0,0,0
LC,42742,472,114,697,42742,1817,42742,1148,1217,754,...,42742,42741,30547,30547,30547,30547,0,0,0,0
NE,7488,319,78,1446,7488,4433,7488,3379,3600,1529,...,7488,7475,3432,3432,3432,3432,16,16,0,0
NT,2534,150,17,89,2534,1404,2534,285,209,87,...,2534,2534,1300,1300,1300,1300,0,0,0,0
VU,684,16,6,25,684,50,684,20,32,24,...,684,684,624,624,624,624,0,0,0,0


In [9]:
df.groupby('higherClassification').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
higherClassification,,,,,,,,,,,,,,,,,,,,,
Sphyrnidae,2,0,0,0,2,2,2,0,2,0,...,0,0,0,0,0,0,0,0,2,0
(Hedw.) Mitt.,1,0,0,0,1,1,1,0,0,1,...,1,1,1,1,1,1,0,0,0,0
(Hook. & Kunth) Spruce ex J.Florsch.,1,0,0,0,1,1,1,0,0,1,...,1,1,1,1,1,1,0,0,0,0
(Hook.) H.A.Crum & Steere,1,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
(Hornsch.) Wijk & Margad.,1,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
| Megaceryle,2,2,2,2,2,2,2,2,0,2,...,0,0,0,0,0,0,0,0,2,0
| Phaethornis,1,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
| Porphyrio,5,5,5,5,5,5,5,5,0,5,...,0,0,0,0,0,0,0,0,5,0


In [10]:
df.groupby('species').count()[["higherClassification"]]

,higherClassification
species,
Abaeis albula,0
Abathomphalus mayaroensis,0
Abelmoschus esculentus,41
Abelmoschus moschatus,10
Ablennes hians,1
...,...
Zygopa michaelis,1
Zygosepalum labiosum,2
Zygostates ligulata,1


In [11]:
df.groupby('species').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
species,,,,,,,,,,,,,,,,,,,,,
Abaeis albula,1,0,0,0,1,1,1,1,1,0,...,1,1,1,1,1,1,0,0,1,0
Abathomphalus mayaroensis,27,0,0,0,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,27,0
Abelmoschus esculentus,42,0,0,0,42,4,42,1,40,2,...,35,35,35,35,35,35,0,0,42,0
Abelmoschus moschatus,12,0,2,0,12,1,12,0,11,2,...,5,5,5,5,5,5,0,0,12,0
Ablennes hians,3,0,0,1,3,3,3,1,0,1,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zygopa michaelis,5,0,0,0,5,1,5,0,1,0,...,0,0,0,0,0,0,0,0,5,0
Zygosepalum labiosum,4,0,0,0,4,1,4,0,1,0,...,1,1,1,1,1,1,0,0,4,0
Zygostates ligulata,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
df.groupby('sex').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
sex,,,,,,,,,,,,,,,,,,,,,
FEMALE,23174,4452,1651,5984,23174,20788,23174,2587,18336,4926,...,7929,7929,7929,7929,7929,7929,2,2,15642,0
MALE,32686,4992,1906,6620,32686,29629,32686,3610,26576,5511,...,9311,9311,9311,9311,9311,9311,2,2,23665,0


In [13]:
df.groupby('taxonRank').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
taxonRank,,,,,,,,,,,,,,,,,,,,,
CLASS,1419,20,21,174,1419,278,1419,29,263,193,...,195,195,195,195,195,195,0,0,1056,0
FAMILY,16230,6164,1891,1241,16230,9040,16230,1067,3306,2310,...,9308,9308,9308,9308,9308,9308,0,0,10053,0
FORM,136,6,17,15,136,46,136,17,81,47,...,30,30,30,30,30,30,0,0,0,0
GENUS,48111,17388,4378,5014,48111,29982,48111,2547,15705,8495,...,25426,25426,25426,25426,25426,25426,0,0,16288,0
KINGDOM,2680,45,145,81,2680,1302,2680,116,1184,236,...,590,590,590,590,590,590,0,0,597,0
ORDER,1056,260,42,85,1056,407,1056,29,116,84,...,375,375,375,375,375,375,0,0,863,0
PHYLUM,1470,44,121,11,1470,281,1470,44,153,129,...,84,84,84,84,84,84,0,0,746,0
SPECIES,457137,22122,33324,27203,457137,202637,457137,97413,227321,65371,...,296559,296559,296557,296557,296556,296556,23,23,456656,0
SUBSPECIES,23600,3794,2259,2984,23600,19607,23600,7157,19467,3461,...,9369,9369,9369,9369,9369,9369,3,3,538,0


In [40]:
df.groupby('publishingCountry').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
publishingCountry,,,,,,,,,,,,,,,,,,,,,
AT,61,0,0,0,61,59,61,2,0,0,...,61,28,28,28,28,0,0,0,0,53
AU,4,0,0,0,4,3,4,0,3,3,...,4,4,4,4,4,0,0,0,0,2
BE,109,0,6,0,109,109,109,6,103,103,...,109,71,71,71,71,0,0,0,0,101
BR,57,0,56,1,57,1,57,0,57,1,...,57,1,1,1,1,0,0,0,0,52
CA,320,303,12,315,320,319,320,5,319,315,...,320,247,247,247,247,0,0,0,0,202
CH,70,0,0,0,70,0,70,66,0,0,...,70,40,40,40,40,25,25,0,0,56
CO,29,0,0,0,29,29,29,0,0,0,...,29,2,2,2,2,0,0,0,0,29
DE,310,0,0,0,310,292,310,14,0,0,...,310,1,1,1,1,0,0,0,0,303
DK,79,0,0,0,79,79,79,0,0,0,...,79,0,0,0,0,0,0,0,0,76


In [20]:
result_df.groupby("kingdomname").count()

,taxonId,taxonname,taxon,kingdom,article
kingdomname,,,,,
Chromista,116,116,116,116,116
Monera,3,3,3,3,3
animal,8562,8562,8562,8562,8562
bacteria,3,3,3,3,3
fungus,612,612,612,612,612
plant,8209,8209,8209,8209,8209
protist,12,12,12,12,12
protozoa,12,12,12,12,12


In [26]:
merged_df[merged_df['taxonId']==""][['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet
5142,7499281.0,,Casearia pitumba,http://www.wikidata.org/entity/Q15389271,Casearia,pitumba
5428,2978125.0,,Copaifera guianensis,http://www.wikidata.org/entity/Q107054779,Copaifera,guianensis
8647,2925401.0,,Avicennia nitida,http://www.wikidata.org/entity/Q87612389,Avicennia,nitida
8656,7715585.0,,Avicennia nitida,http://www.wikidata.org/entity/Q87612389,Avicennia,nitida
8969,5248141.0,,Polyporus fomentarius,http://www.wikidata.org/entity/Q105065407,Polyporus,fomentarius
...,...,...,...,...,...,...
466793,7285715.0,,Heteropterys suberosa,http://www.wikidata.org/entity/Q111094489,Heteropterys,suberosa
466823,8375315.0,,Auplopus stagei,http://www.wikidata.org/entity/Q117823629,Auplopus,stagei
466842,2717425.0,,Eleocharis ochreata,http://www.wikidata.org/entity/Q110437399,Eleocharis,ochreata
466865,5565418.0,,Myrica javanica,http://www.wikidata.org/entity/Q50844166,Myrica,javanica


In [16]:
    wdtaxqid

{'2241784': 'http://www.wikidata.org/entity/Q3016830',
 '3992233': 'http://www.wikidata.org/entity/Q15334447',
 '2514762': 'http://www.wikidata.org/entity/Q2117089',
 '2485721': 'http://www.wikidata.org/entity/Q1270918',
 '2286513': 'http://www.wikidata.org/entity/Q105972189',
 '2929028': 'http://www.wikidata.org/entity/Q15539093',
 '3086328': 'http://www.wikidata.org/entity/Q15606320',
 '7304240': 'http://www.wikidata.org/entity/Q15609790',
 '5415073': 'http://www.wikidata.org/entity/Q17682523',
 '5253985': 'http://www.wikidata.org/entity/Q10600153',
 '2221803': 'http://www.wikidata.org/entity/Q4470463',
 '2352597': 'http://www.wikidata.org/entity/Q6405418',
 '6004674': 'http://www.wikidata.org/entity/Q14929871',
 '2900288': 'http://www.wikidata.org/entity/Q7396079',
 '2681226': 'http://www.wikidata.org/entity/Q17199991',
 '3823448': 'http://www.wikidata.org/entity/Q110583725',
 '1837540': 'http://www.wikidata.org/entity/Q13632704',
 '3055755': 'http://www.wikidata.org/entity/Q1537070

In [29]:
df_image.groupby("license").count()

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,rightsHolder
license,,,,,,,,,,,,,,
(c) Field Museum of Natural History,24,24,24,24,0,24,24,0,0,23,24,0,0,24
(c) Field Museum of Natural History - CC BY-NC 4.0,3396,3396,3396,3396,0,3396,3396,0,0,3124,3366,0,662,3396
Attribution-ShareAlike (BY-SA) Creative Commons License and GNU Free Documentation License (GFDL),408,408,408,408,408,408,0,0,0,0,408,0,0,408
"Copyright Borror Lab of Bioacoustics, Ohio State University 2018",1,1,1,1,0,1,1,0,0,0,0,0,0,1
Copyright permission not set,10,10,10,10,0,0,4,0,0,0,10,0,10,0
Creative Commons,13,0,0,0,13,0,0,0,0,0,0,0,0,13
Daniel Barthelemy (cc-by-nc),120,120,120,120,0,120,120,120,0,120,120,0,0,120
"Partial images provided by this server are released under the Creative Commons cc-by-sa 3.0 (generic) licence [https://creativecommons.org/licenses/by-sa/3.0/de/]. Please credit images to BGBM following our citation guidelines [https://ww2.bgbm.org/Herbarium/cite.cfm]. If you would like to use images in a format or resolution which is not provided here, please contact us (d.roepert[at]bgbm.org).",313,165,313,313,313,0,0,0,0,0,0,0,0,0
Usage Conditions Apply,2952,2952,2952,2952,0,2952,2208,2223,0,0,2843,0,2952,0


In [30]:
df_image.groupby("type").count()

,gbifID,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
type,,,,,,,,,,,,,,
InteractiveResource,4487,4487,4487,0,4461,4487,0,0,0,4487,0,0,4487,4487
Sound,1314,1314,1314,6,1,1303,0,0,6,1313,0,6,1314,1314
StillImage,138453,138446,138079,18099,33464,83748,13513,0,15358,63244,0,26913,130308,111377


In [31]:
df_image[df_image["type"]=="Sound"]

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
8407,2243677327,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,61 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8412,2243677692,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,142 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8415,2243677773,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,43 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8416,2243677804,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,52 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8419,2243677831,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,25 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151524,4173214802,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,19 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151525,4173215545,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,4 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151530,4173216823,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,12 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151532,4173217552,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,7 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot


In [22]:
df_image.groupby('identifier').count()

,gbifID,type,format,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
identifier,,,,,,,,,,,,,,
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00250008.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00250662.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00251190.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00400057.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00539887.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://zenodo.org/record/2702457/files/CAM021233_v.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0
https://zenodo.org/record/2702457/files/CAM021234_d.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0
https://zenodo.org/record/2702457/files/CAM021234_v.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0


In [71]:
len(gbifRDF.all_nodes())

0

In [ ]:
len (gbifRDF.all_nodes())

In [33]:
query = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?taxon ?taxonLabel WHERE {
           ?observation wdt:P225 ?taxon .
           ?taxon ?p ?taxonLabel .
            }"""
results = gbifRDF.query(query)
for row in results:
    print(row["taxon"], row["taxonLabel"])
    mdFile = mdutils.MdUtils(file_name='taxa/'+ row["taxonLabel"].replace(" ", "_"), title=row["taxonLabel"]+" ("+row["taxon"]+")")
    if URIRef(row["taxon"]) in taxoninformation.keys():
        if "wikiintroduction" in taxoninformation[URIRef(row["taxon"])].keys():
            mdFile.new_line(taxoninformation[URIRef(row["taxon"])]["wikiintroduction"])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('' , '_')})")
        mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{row['taxon'].replace('http://www.wikidata.org/entity/', '')})")

http://www.wikidata.org/entity/Q15381185 Sauvagesia erecta


NameError: name 'mdutils' is not defined

In [34]:
import mdutils
for taxon in table.keys():
    if table[taxon]["taxon_name"] in verified:
        mdFile = mdutils.MdUtils(file_name='new_articles/'+table[taxon]["taxon_name"].replace(" ", "_"),title=table[taxon]["taxon_name"])
        mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
        for image in table[taxon]["photos"]:
            mdFile.new_line(f"![{table[taxon]['taxon_name']}]({image.replace('square', 'medium')})")
        mdFile.create_md_file()

NameError: name 'table' is not defined